In [ ]:
!pip install wandb
!wandb login 

In [ ]:
LEARNING_RATES = [0.005, 0.003, 0.001, 0.007] # best = 0.003
# best batch size = 128 
# works better with less encoder layers, more encoder filters if z_dim is kept constant
# batch normalization always makes a big difference
# leaky_relu > tanh ~~ relu

## Prelim stuff

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, Conv2DTranspose, Dense, Flatten, Reshape, BatchNormalization, Lambda, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from PIL import Image, ImageOps
from skimage.metrics import structural_similarity as ssim

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import os

from tensorflow.keras.datasets import fashion_mnist, cifar10

In [ ]:
import wandb
from wandb.keras import WandbCallback

In [ ]:
input_sizes_dict = {
    'm': 28,
    'c': 32,
}
channels_dict = {
    'm': 1,
    'c': 3,
}

In [ ]:
def fit_model_and_log_metrics(run_name, run_notes, model, epochs, batch_size):
    run = wandb.init(project="autoencoders_RQ_comparison", entity="csc2231-yash-siphelele", name=run_name, notes=run_notes)
    wandb_callback = WandbCallback(
                monitor="val_loss", verbose=0, mode="auto", save_weights_only=(False),
                log_weights=(False), log_gradients=(False), save_model=(True),
                training_data=None, validation_data=None, labels=[], predictions=36,
                generator=None, input_type=None, output_type=None, log_evaluation=(False),
                validation_steps=None, class_colors=None, log_batch_frequency=None,
                log_best_prefix="best_", save_graph=(False), validation_indexes=None,
                validation_row_processor=None, prediction_row_processor=None,
                infer_missing_processors=(False), log_evaluation_frequency=0)   
    early_stopping_callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=5, mode='auto')
    history = model.fit(x_train, x_train, epochs = epochs, batch_size = batch_size, validation_data = (x_test, x_test), callbacks=[wandb_callback, early_stopping_callback]) 
    decoded_imgs = model.predict(x_test)
    ssim_scores = []
    for i in range(len(x_test)):
        if CURRENT_DATASET_TYPE == 'c':
            ssim_value = ssim(x_test[i].reshape(input_sizes_dict[CURRENT_DATASET_TYPE], input_sizes_dict[CURRENT_DATASET_TYPE], channels_dict[CURRENT_DATASET_TYPE]), 
                                            decoded_imgs[i].reshape(input_sizes_dict[CURRENT_DATASET_TYPE], input_sizes_dict[CURRENT_DATASET_TYPE], channels_dict[CURRENT_DATASET_TYPE]),
                              multichannel=True)
        else:
            ssim_value = ssim(x_test[i].reshape(input_sizes_dict[CURRENT_DATASET_TYPE], input_sizes_dict[CURRENT_DATASET_TYPE]), 
                                            decoded_imgs[i].reshape(input_sizes_dict[CURRENT_DATASET_TYPE], input_sizes_dict[CURRENT_DATASET_TYPE]))
        ssim_scores.append(ssim_value)

    avg_ssim_score_x_test = np.mean(ssim_scores)
    wandb.log({"avg_ssim_score_x_test": avg_ssim_score_x_test})
    print("Avg SSIM over X_test:", avg_ssim_score_x_test)
    # model.save(os.path.join(wandb.run.dir, "model.h5"))
    run.finish()

In [ ]:
# (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
# x_train = x_train.astype('float32') / 255
# x_test = x_test.astype('float32') / 255
# x_train.shape, x_test.shape, y_train.shape, y_test.shape 

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_train.shape, x_test.shape, y_train.shape, y_test.shape 

170508288/170498071 [==============================] - 3s 0us/step


((50000, 32, 32, 3), (10000, 32, 32, 3), (50000, 1), (10000, 1))

In [ ]:
Z_DIM = [20, 100, 300, 5000]  # Dimension of the latent vector (z), model is trained using different values of Z_DIM

In [ ]:
CURRENT_DATASET_TYPE = 'c'

## Experiments

### Exp 1.0

In [ ]:
'''
Model vae_1.0
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

x = Conv2D(4, (3, 3), activation='relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
# print(shape_before_flattening)
x = Flatten()(x)

z_mu = Dense(latent_dim)(x)
z_log_sigma = Dense(latent_dim, kernel_initializer='zeros',bias_initializer='zeros')(x)


# sampling function
def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon

# sample vector from the latent distribution
z = Lambda(sampling)([z_mu, z_log_sigma])

encoder = Model(input_img,z)

# decoder takes the latent distribution sample as input
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(4096, activation='relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
x = Reshape((8,8,64))(x)

x = Conv2DTranspose(32, (3, 3),strides= 2, padding='same')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(16, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2DTranspose(3, (3, 3),strides= 2, padding='same', activation='sigmoid')(x)


# decoder model statement
decoder = Model(decoder_input, x)

# apply the decoder to the sample from the latent distribution
pred = decoder(z)


def vae_loss(x, pred):
    x = K.flatten(x)
    pred = K.flatten(pred)
    # Reconstruction loss
    #reconst_loss = 100 * binary_crossentropy(x, pred)
    reconst_loss = 1000*K.mean(K.square(x - pred))
    
    # KL divergence
    kl_loss = -0.5 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
    
    return (reconst_loss + kl_loss)


# VAE model statement
vae = Model(input_img, pred)
vae.add_loss(vae_loss(input_img,pred))
optimizer = Adam(learning_rate=0.0005)
vae.compile(optimizer=optimizer, loss=None)

vae.summary()


# fit and track the model 
fit_model_and_log_metrics("Model vae_1.0_c", "With maxpooling, convtranspose, dense, and flatten", vae, 50, 256)

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_9 (Conv2D)              (None, 16, 16, 32)   896         ['input_7[0][0]']                
                                                                                                  
 batch_normalization_12 (BatchN  (None, 16, 16, 32)  128         ['conv2d_9[0][0]']               
 ormalization)                                                                                    
                                                                                                  
 conv2d_10 (Conv2D)             (None, 8, 8, 16)     4624        ['batch_normalization_12[0

epoch,▁▂▃▅▆▇█
loss,█▃▂▁▁▁▁
val_loss,█▅▂▂▁▁▁
best_epoch,6
best_val_loss,11.25468
epoch,6
loss,10.68629
val_loss,11.25468


Epoch 1/50
196/196 [==============================] - 7s 29ms/step - loss: 28.6772 - val_loss: 36.3252 - _timestamp: 1648694211.0000 - _runtime: 13.0000
Epoch 2/50
196/196 [==============================] - 5s 24ms/step - loss: 14.7436 - val_loss: 21.9665 - _timestamp: 1648694216.0000 - _runtime: 18.0000
Epoch 3/50
196/196 [==============================] - 5s 24ms/step - loss: 12.1907 - val_loss: 15.7596 - _timestamp: 1648694221.0000 - _runtime: 23.0000
Epoch 4/50
196/196 [==============================] - 5s 25ms/step - loss: 11.2775 - val_loss: 12.5467 - _timestamp: 1648694226.0000 - _runtime: 28.0000
Epoch 5/50
196/196 [==============================] - 5s 24ms/step - loss: 10.9009 - val_loss: 11.2220 - _timestamp: 1648694230.0000 - _runtime: 32.0000
Epoch 6/50
196/196 [==============================] - 5s 25ms/step - loss: 10.6780 - val_loss: 10.9920 - _timestamp: 1648694235.0000 - _runtime: 37.0000
Epoch 7/50
196/196 [==============================] - 5s 24ms/step - loss: 10.5318

avg_ssim_score_x_test,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_ssim_score_x_test,0.7111
best_epoch,48
best_val_loss,9.62338
epoch,49
loss,9.37728
val_loss,9.64616


### Exp 1.1

In [ ]:
'''
Model vae_1.1
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(input_img)
# x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
# x = BatchNormalization()(x)

x = Conv2D(4, (3, 3), activation='relu', padding='same', strides=2)(x)
# x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
# print(shape_before_flattening)
x = Flatten()(x)

z_mu = Dense(latent_dim)(x)
z_log_sigma = Dense(latent_dim, kernel_initializer='zeros', bias_initializer='zeros')(x)


# sampling function
def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon

# sample vector from the latent distribution
z = Lambda(sampling)([z_mu, z_log_sigma])

encoder = Model(input_img,z)

# decoder takes the latent distribution sample as input
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(4096, activation='relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
x = Reshape((8,8,64))(x)

x = Conv2DTranspose(32, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2DTranspose(16, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2DTranspose(3, (3, 3),strides= 2, padding='same', activation='sigmoid')(x)


# decoder model statement
decoder = Model(decoder_input, x)

# apply the decoder to the sample from the latent distribution
pred = decoder(z)


def vae_loss(x, pred):
    x = K.flatten(x)
    pred = K.flatten(pred)
    # Reconstruction loss
    #reconst_loss = 100 * binary_crossentropy(x, pred)
    reconst_loss = 1000*K.mean(K.square(x - pred))
    
    # KL divergence
    kl_loss = -0.5 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
    
    return (reconst_loss + kl_loss)


# VAE model statement
vae = Model(input_img, pred)
vae.add_loss(vae_loss(input_img,pred))
optimizer = Adam(learning_rate=0.0005)
vae.compile(optimizer=optimizer, loss=None)

vae.summary()


# fit and track the model 
fit_model_and_log_metrics("Model vae_1.1_c", "With maxpooling, convtranspose, dense, and flatten", vae, 50, 256)

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_12 (Conv2D)             (None, 16, 16, 32)   896         ['input_9[0][0]']                
                                                                                                  
 conv2d_13 (Conv2D)             (None, 8, 8, 16)     4624        ['conv2d_12[0][0]']              
                                                                                                  
 conv2d_14 (Conv2D)             (None, 4, 4, 4)      580         ['conv2d_13[0][0]']              
                                                                                           

Epoch 1/50
196/196 [==============================] - 6s 26ms/step - loss: 33.6735 - val_loss: 19.4861 - _timestamp: 1648694622.0000 - _runtime: 12.0000
Epoch 2/50
196/196 [==============================] - 5s 24ms/step - loss: 17.8745 - val_loss: 16.6308 - _timestamp: 1648694627.0000 - _runtime: 17.0000
Epoch 3/50
196/196 [==============================] - 4s 22ms/step - loss: 15.2202 - val_loss: 13.8489 - _timestamp: 1648694631.0000 - _runtime: 21.0000
Epoch 4/50
196/196 [==============================] - 5s 23ms/step - loss: 13.3206 - val_loss: 13.1313 - _timestamp: 1648694636.0000 - _runtime: 26.0000
Epoch 5/50
196/196 [==============================] - 5s 24ms/step - loss: 12.7703 - val_loss: 13.0450 - _timestamp: 1648694641.0000 - _runtime: 31.0000
Epoch 6/50
196/196 [==============================] - 4s 23ms/step - loss: 12.3767 - val_loss: 12.2148 - _timestamp: 1648694645.0000 - _runtime: 35.0000
Epoch 7/50
196/196 [==============================] - 4s 23ms/step - loss: 12.0049

avg_ssim_score_x_test,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_ssim_score_x_test,0.70896
best_epoch,48
best_val_loss,9.73552
epoch,49
loss,9.55519
val_loss,9.75746


## Exp 2

In [ ]:
'''
Model vae_2
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
# print(shape_before_flattening)
x = Flatten()(x)

z_mu = Dense(latent_dim)(x)
z_log_sigma = Dense(latent_dim, kernel_initializer='zeros', bias_initializer='zeros')(x)


# sampling function
def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon

# sample vector from the latent distribution
z = Lambda(sampling)([z_mu, z_log_sigma])

encoder = Model(input_img,z)

# decoder takes the latent distribution sample as input
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(4096, activation='relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
x = Reshape((8,8,64))(x)

x = Conv2DTranspose(32, (3, 3),strides= 2, padding='same')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(16, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2DTranspose(3, (3, 3),strides= 2, padding='same', activation='sigmoid')(x)


# decoder model statement
decoder = Model(decoder_input, x)

# apply the decoder to the sample from the latent distribution
pred = decoder(z)


def vae_loss(x, pred):
    x = K.flatten(x)
    pred = K.flatten(pred)
    # Reconstruction loss
    #reconst_loss = 100 * binary_crossentropy(x, pred)
    reconst_loss = 1000*K.mean(K.square(x - pred))
    
    # KL divergence
    kl_loss = -0.5 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
    
    return (reconst_loss + kl_loss)


# VAE model statement
vae = Model(input_img, pred)
vae.add_loss(vae_loss(input_img,pred))
optimizer = Adam(learning_rate=0.0005)
vae.compile(optimizer=optimizer, loss=None)

vae.summary()


# fit and track the model 
fit_model_and_log_metrics("Model vae_2_c", "With maxpooling, convtranspose, dense, and flatten", vae, 50, 256)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_3 (Conv2D)              (None, 16, 16, 32)   896         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_4 (BatchNo  (None, 16, 16, 32)  128         ['conv2d_3[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 conv2d_4 (Conv2D)              (None, 8, 8, 16)     4624        ['batch_normalization_4[0][

wandb: Currently logged in as: csc2231-yash-siphelele (use `wandb login --relogin` to force relogin)


Epoch 1/50
196/196 [==============================] - 15s 26ms/step - loss: 24.1130 - val_loss: 44.4906 - _timestamp: 1648819958.0000 - _runtime: 21.0000
Epoch 2/50
196/196 [==============================] - 4s 23ms/step - loss: 11.5708 - val_loss: 24.3547 - _timestamp: 1648819962.0000 - _runtime: 25.0000
Epoch 3/50
196/196 [==============================] - 4s 22ms/step - loss: 9.5190 - val_loss: 11.6109 - _timestamp: 1648819967.0000 - _runtime: 30.0000
Epoch 4/50
196/196 [==============================] - 4s 22ms/step - loss: 8.5544 - val_loss: 9.4180 - _timestamp: 1648819971.0000 - _runtime: 34.0000
Epoch 5/50
196/196 [==============================] - 4s 22ms/step - loss: 8.1158 - val_loss: 9.1853 - _timestamp: 1648819975.0000 - _runtime: 38.0000
Epoch 6/50
196/196 [==============================] - 4s 22ms/step - loss: 7.8196 - val_loss: 8.2644 - _timestamp: 1648819980.0000 - _runtime: 43.0000
Epoch 7/50
196/196 [==============================] - 4s 22ms/step - loss: 7.6139 - val_

avg_ssim_score_x_test,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_ssim_score_x_test,0.81377
best_epoch,48
best_val_loss,6.45458
epoch,49
loss,6.3029
val_loss,6.70844


## Exp 2.1

In [ ]:
'''
Model vae_2.1
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
# print(shape_before_flattening)
x = Flatten()(x)

z_mu = Dense(latent_dim)(x)
z_log_sigma = Dense(latent_dim, kernel_initializer='zeros', bias_initializer='zeros')(x)


# sampling function
def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon

# sample vector from the latent distribution
z = Lambda(sampling)([z_mu, z_log_sigma])

encoder = Model(input_img,z)

# decoder takes the latent distribution sample as input
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(4096, activation='relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
x = Reshape((8,8,64))(x)

x = Conv2DTranspose(32, (3, 3),strides= 2, padding='same')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(16, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2DTranspose(3, (3, 3),strides= 2, padding='same', activation='sigmoid')(x)


# decoder model statement
decoder = Model(decoder_input, x)

# apply the decoder to the sample from the latent distribution
pred = decoder(z)


def vae_loss(x, pred):
    x = K.flatten(x)
    pred = K.flatten(pred)
    # Reconstruction loss
    #reconst_loss = 100 * binary_crossentropy(x, pred)
    reconst_loss = 1000*K.mean(K.square(x - pred))
    
    # KL divergence
    kl_loss = -0.5 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
    
    return (reconst_loss + kl_loss)


# VAE model statement
vae = Model(input_img, pred)
vae.add_loss(vae_loss(input_img,pred))
optimizer = Adam(learning_rate=0.0005)
vae.compile(optimizer=optimizer, loss=None)

vae.summary()


# fit and track the model 
fit_model_and_log_metrics("Model vae_2.1_c", "With maxpooling, convtranspose, dense, and flatten", vae, 50, 256)

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_6 (Conv2D)              (None, 16, 16, 32)   896         ['input_5[0][0]']                
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 16, 16, 32)  128         ['conv2d_6[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 conv2d_7 (Conv2D)              (None, 8, 8, 16)     4624        ['batch_normalization_8[0][

Epoch 1/50
196/196 [==============================] - 6s 25ms/step - loss: 23.7102 - val_loss: 34.1079 - _timestamp: 1648820318.0000 - _runtime: 12.0000
Epoch 2/50
196/196 [==============================] - 5s 24ms/step - loss: 10.8032 - val_loss: 16.0269 - _timestamp: 1648820322.0000 - _runtime: 16.0000
Epoch 3/50
196/196 [==============================] - 4s 22ms/step - loss: 8.4434 - val_loss: 12.0435 - _timestamp: 1648820327.0000 - _runtime: 21.0000
Epoch 4/50
196/196 [==============================] - 4s 22ms/step - loss: 7.3104 - val_loss: 8.3705 - _timestamp: 1648820331.0000 - _runtime: 25.0000
Epoch 5/50
196/196 [==============================] - 4s 22ms/step - loss: 6.7593 - val_loss: 7.5904 - _timestamp: 1648820335.0000 - _runtime: 29.0000
Epoch 6/50
196/196 [==============================] - 5s 24ms/step - loss: 6.3568 - val_loss: 6.7037 - _timestamp: 1648820340.0000 - _runtime: 34.0000
Epoch 7/50
196/196 [==============================] - 4s 22ms/step - loss: 6.0302 - val_l

avg_ssim_score_x_test,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_ssim_score_x_test,0.88439
best_epoch,46
best_val_loss,4.41076
epoch,49
loss,4.30456
val_loss,4.50556


## Exp 3

In [ ]:
'''
Model vae_3
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
# print(shape_before_flattening)
x = Flatten()(x)

z_mu = Dense(latent_dim)(x)
z_log_sigma = Dense(latent_dim, kernel_initializer='zeros', bias_initializer='zeros')(x)


# sampling function
def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon

# sample vector from the latent distribution
z = Lambda(sampling)([z_mu, z_log_sigma])

encoder = Model(input_img,z)

# decoder takes the latent distribution sample as input
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(4096, activation='relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
x = Reshape((8,8,64))(x)

x = Conv2DTranspose(32, (3, 3),strides= 2, padding='same')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(16, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2DTranspose(3, (3, 3),strides= 2, padding='same', activation='sigmoid')(x)


# decoder model statement
decoder = Model(decoder_input, x)

# apply the decoder to the sample from the latent distribution
pred = decoder(z)


def vae_loss(x, pred):
    x = K.flatten(x)
    pred = K.flatten(pred)
    # Reconstruction loss
    #reconst_loss = 100 * binary_crossentropy(x, pred)
    reconst_loss = 1000*K.mean(K.square(x - pred))
    
    # KL divergence
    kl_loss = -0.5 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
    
    return (reconst_loss + kl_loss)


# VAE model statement
vae = Model(input_img, pred)
vae.add_loss(vae_loss(input_img,pred))
optimizer = Adam(learning_rate=LEARNING_RATES[1])
vae.compile(optimizer=optimizer, loss=None)

vae.summary()


# fit and track the model 
fit_model_and_log_metrics("Model vae_3_c", "With maxpooling, convtranspose, dense, and flatten", vae, 50, 256)

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_9 (Conv2D)              (None, 16, 16, 32)   896         ['input_7[0][0]']                
                                                                                                  
 batch_normalization_12 (BatchN  (None, 16, 16, 32)  128         ['conv2d_9[0][0]']               
 ormalization)                                                                                    
                                                                                                  
 conv2d_10 (Conv2D)             (None, 8, 8, 16)     4624        ['batch_normalization_12[0

Epoch 1/50
196/196 [==============================] - 6s 25ms/step - loss: 16.8039 - val_loss: 32.1823 - _timestamp: 1648820897.0000 - _runtime: 12.0000
Epoch 2/50
196/196 [==============================] - 4s 22ms/step - loss: 8.0690 - val_loss: 21.4717 - _timestamp: 1648820901.0000 - _runtime: 16.0000
Epoch 3/50
196/196 [==============================] - 5s 23ms/step - loss: 6.4910 - val_loss: 11.1096 - _timestamp: 1648820905.0000 - _runtime: 20.0000
Epoch 4/50
196/196 [==============================] - 4s 22ms/step - loss: 5.7949 - val_loss: 8.5129 - _timestamp: 1648820910.0000 - _runtime: 25.0000
Epoch 5/50
196/196 [==============================] - 4s 22ms/step - loss: 5.3870 - val_loss: 7.2107 - _timestamp: 1648820914.0000 - _runtime: 29.0000
Epoch 6/50
196/196 [==============================] - 4s 22ms/step - loss: 5.1019 - val_loss: 6.8016 - _timestamp: 1648820918.0000 - _runtime: 33.0000
Epoch 7/50
196/196 [==============================] - 4s 22ms/step - loss: 4.9217 - val_lo

avg_ssim_score_x_test,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_ssim_score_x_test,0.89228
best_epoch,46
best_val_loss,4.12265
epoch,49
loss,3.92259
val_loss,4.13581


## Exp 3.1

In [ ]:
'''
Model vae_3.1
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
# print(shape_before_flattening)
x = Flatten()(x)

z_mu = Dense(latent_dim)(x)
z_log_sigma = Dense(latent_dim, kernel_initializer='zeros', bias_initializer='zeros')(x)


# sampling function
def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon

# sample vector from the latent distribution
z = Lambda(sampling)([z_mu, z_log_sigma])

encoder = Model(input_img,z)

# decoder takes the latent distribution sample as input
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(4096, activation='relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
x = Reshape((8,8,64))(x)

x = Conv2DTranspose(32, (3, 3),strides= 2, padding='same')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(16, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2DTranspose(3, (3, 3),strides= 2, padding='same', activation='sigmoid')(x)


# decoder model statement
decoder = Model(decoder_input, x)

# apply the decoder to the sample from the latent distribution
pred = decoder(z)


def vae_loss(x, pred):
    x = K.flatten(x)
    pred = K.flatten(pred)
    # Reconstruction loss
    #reconst_loss = 100 * binary_crossentropy(x, pred)
    reconst_loss = 1000*K.mean(K.square(x - pred))
    
    # KL divergence
    kl_loss = -0.5 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
    
    return (reconst_loss + kl_loss)


# VAE model statement
vae = Model(input_img, pred)
vae.add_loss(vae_loss(input_img,pred))
optimizer = Adam(learning_rate=LEARNING_RATES[2])
vae.compile(optimizer=optimizer, loss=None)

vae.summary()


# fit and track the model 
fit_model_and_log_metrics("Model vae_3.1_c", "With maxpooling, convtranspose, dense, and flatten", vae, 50, 256)

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_12 (Conv2D)             (None, 16, 16, 32)   896         ['input_9[0][0]']                
                                                                                                  
 batch_normalization_16 (BatchN  (None, 16, 16, 32)  128         ['conv2d_12[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_13 (Conv2D)             (None, 8, 8, 16)     4624        ['batch_normalization_16[0

Epoch 1/50
196/196 [==============================] - 6s 25ms/step - loss: 19.1197 - val_loss: 34.4375 - _timestamp: 1648821653.0000 - _runtime: 13.0000
Epoch 2/50
196/196 [==============================] - 4s 22ms/step - loss: 8.8710 - val_loss: 13.4680 - _timestamp: 1648821657.0000 - _runtime: 17.0000
Epoch 3/50
196/196 [==============================] - 4s 22ms/step - loss: 6.9381 - val_loss: 8.2066 - _timestamp: 1648821662.0000 - _runtime: 22.0000
Epoch 4/50
196/196 [==============================] - 4s 22ms/step - loss: 6.2041 - val_loss: 7.4096 - _timestamp: 1648821666.0000 - _runtime: 26.0000
Epoch 5/50
196/196 [==============================] - 4s 22ms/step - loss: 5.7721 - val_loss: 6.0638 - _timestamp: 1648821670.0000 - _runtime: 30.0000
Epoch 6/50
196/196 [==============================] - 4s 22ms/step - loss: 5.5193 - val_loss: 5.7735 - _timestamp: 1648821675.0000 - _runtime: 35.0000
Epoch 7/50
196/196 [==============================] - 4s 22ms/step - loss: 5.3204 - val_los

avg_ssim_score_x_test,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_ssim_score_x_test,0.88768
best_epoch,48
best_val_loss,4.30267
epoch,49
loss,4.19216
val_loss,4.33078


## Exp 3.2

In [ ]:
'''
Model vae_3.2
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
# print(shape_before_flattening)
x = Flatten()(x)

z_mu = Dense(latent_dim)(x)
z_log_sigma = Dense(latent_dim, kernel_initializer='zeros', bias_initializer='zeros')(x)


# sampling function
def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon

# sample vector from the latent distribution
z = Lambda(sampling)([z_mu, z_log_sigma])

encoder = Model(input_img,z)

# decoder takes the latent distribution sample as input
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(4096, activation='relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
x = Reshape((8,8,64))(x)

x = Conv2DTranspose(32, (3, 3),strides= 2, padding='same')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(16, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2DTranspose(3, (3, 3),strides= 2, padding='same', activation='sigmoid')(x)


# decoder model statement
decoder = Model(decoder_input, x)

# apply the decoder to the sample from the latent distribution
pred = decoder(z)


def vae_loss(x, pred):
    x = K.flatten(x)
    pred = K.flatten(pred)
    # Reconstruction loss
    #reconst_loss = 100 * binary_crossentropy(x, pred)
    reconst_loss = 1000*K.mean(K.square(x - pred))
    
    # KL divergence
    kl_loss = -0.5 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
    
    return (reconst_loss + kl_loss)


# VAE model statement
vae = Model(input_img, pred)
vae.add_loss(vae_loss(input_img,pred))
optimizer = Adam(learning_rate=LEARNING_RATES[3])
vae.compile(optimizer=optimizer, loss=None)

vae.summary()


# fit and track the model 
fit_model_and_log_metrics("Model vae_3.2_c", "With maxpooling, convtranspose, dense, and flatten", vae, 50, 256)

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_15 (Conv2D)             (None, 16, 16, 32)   896         ['input_11[0][0]']               
                                                                                                  
 batch_normalization_20 (BatchN  (None, 16, 16, 32)  128         ['conv2d_15[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_16 (Conv2D)             (None, 8, 8, 16)     4624        ['batch_normalization_20[0

Epoch 1/50
196/196 [==============================] - 6s 26ms/step - loss: 18.4445 - val_loss: 25.2537 - _timestamp: 1648821941.0000 - _runtime: 13.0000
Epoch 2/50
196/196 [==============================] - 4s 22ms/step - loss: 8.5525 - val_loss: 19.2325 - _timestamp: 1648821945.0000 - _runtime: 17.0000
Epoch 3/50
196/196 [==============================] - 4s 22ms/step - loss: 6.7046 - val_loss: 14.6872 - _timestamp: 1648821950.0000 - _runtime: 22.0000
Epoch 4/50
196/196 [==============================] - 4s 22ms/step - loss: 5.8743 - val_loss: 8.7561 - _timestamp: 1648821954.0000 - _runtime: 26.0000
Epoch 5/50
196/196 [==============================] - 4s 22ms/step - loss: 5.4547 - val_loss: 8.3813 - _timestamp: 1648821958.0000 - _runtime: 30.0000
Epoch 6/50
196/196 [==============================] - 4s 22ms/step - loss: 5.1733 - val_loss: 7.1755 - _timestamp: 1648821963.0000 - _runtime: 35.0000
Epoch 7/50
196/196 [==============================] - 4s 21ms/step - loss: 4.9781 - val_lo

avg_ssim_score_x_test,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
loss,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▄▂▂▂▂▂▂▂▁▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_ssim_score_x_test,0.88863
best_epoch,30
best_val_loss,4.40704
epoch,40
loss,3.93889
val_loss,4.52141


## Exp 4

In [ ]:
'''
Model vae_4
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

# x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
# x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
# print(shape_before_flattening)
x = Flatten()(x)

z_mu = Dense(latent_dim)(x)
z_log_sigma = Dense(latent_dim, kernel_initializer='zeros', bias_initializer='zeros')(x)


# sampling function
def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon

# sample vector from the latent distribution
z = Lambda(sampling)([z_mu, z_log_sigma])

encoder = Model(input_img,z)

# decoder takes the latent distribution sample as input
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(4096, activation='relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
x = Reshape((8,8,64))(x)

x = Conv2DTranspose(32, (3, 3),strides= 2, padding='same')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(16, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2DTranspose(3, (3, 3),strides= 2, padding='same', activation='sigmoid')(x)


# decoder model statement
decoder = Model(decoder_input, x)

# apply the decoder to the sample from the latent distribution
pred = decoder(z)


def vae_loss(x, pred):
    x = K.flatten(x)
    pred = K.flatten(pred)
    # Reconstruction loss
    #reconst_loss = 100 * binary_crossentropy(x, pred)
    reconst_loss = 1000*K.mean(K.square(x - pred))
    
    # KL divergence
    kl_loss = -0.5 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
    
    return (reconst_loss + kl_loss)


# VAE model statement
vae = Model(input_img, pred)
vae.add_loss(vae_loss(input_img,pred))
optimizer = Adam(learning_rate=LEARNING_RATES[1])
vae.compile(optimizer=optimizer, loss=None)

vae.summary()


# fit and track the model 
fit_model_and_log_metrics("Model vae_4_c", "With maxpooling, convtranspose, dense, and flatten", vae, 50, 256)

Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_18 (Conv2D)             (None, 16, 16, 32)   896         ['input_13[0][0]']               
                                                                                                  
 batch_normalization_24 (BatchN  (None, 16, 16, 32)  128         ['conv2d_18[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_19 (Conv2D)             (None, 8, 8, 16)     4624        ['batch_normalization_24[0

Epoch 1/50
196/196 [==============================] - 7s 26ms/step - loss: 14.7957 - val_loss: 36.7034 - _timestamp: 1648822547.0000 - _runtime: 16.0000
Epoch 2/50
196/196 [==============================] - 4s 22ms/step - loss: 6.1991 - val_loss: 13.6288 - _timestamp: 1648822551.0000 - _runtime: 20.0000
Epoch 3/50
196/196 [==============================] - 5s 24ms/step - loss: 5.1441 - val_loss: 7.3141 - _timestamp: 1648822556.0000 - _runtime: 25.0000
Epoch 4/50
196/196 [==============================] - 4s 22ms/step - loss: 4.6637 - val_loss: 5.8296 - _timestamp: 1648822560.0000 - _runtime: 29.0000
Epoch 5/50
196/196 [==============================] - 4s 22ms/step - loss: 4.3929 - val_loss: 5.0025 - _timestamp: 1648822564.0000 - _runtime: 33.0000
Epoch 6/50
196/196 [==============================] - 4s 22ms/step - loss: 4.1637 - val_loss: 4.7270 - _timestamp: 1648822569.0000 - _runtime: 38.0000
Epoch 7/50
196/196 [==============================] - 5s 24ms/step - loss: 4.0491 - val_los

avg_ssim_score_x_test,▁
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_ssim_score_x_test,0.9146
best_epoch,19
best_val_loss,3.68062
epoch,29
loss,3.51297
val_loss,3.71067


## Exp 5

In [ ]:
'''
Model vae_5
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

# x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
# x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
# print(shape_before_flattening)
x = Flatten()(x)

z_mu = Dense(latent_dim)(x)
z_log_sigma = Dense(latent_dim, kernel_initializer='zeros', bias_initializer='zeros')(x)


# sampling function
def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon

# sample vector from the latent distribution
z = Lambda(sampling)([z_mu, z_log_sigma])

encoder = Model(input_img,z)

# decoder takes the latent distribution sample as input
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(4096, activation='relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
x = Reshape((8,8,64))(x)

x = Conv2DTranspose(32, (3, 3),strides= 2, padding='same')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(16, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2DTranspose(3, (3, 3),strides= 2, padding='same', activation='sigmoid')(x)


# decoder model statement
decoder = Model(decoder_input, x)

# apply the decoder to the sample from the latent distribution
pred = decoder(z)


def vae_loss(x, pred):
    x = K.flatten(x)
    pred = K.flatten(pred)
    # Reconstruction loss
    #reconst_loss = 100 * binary_crossentropy(x, pred)
    reconst_loss = 1000*K.mean(K.square(x - pred))
    
    # KL divergence
    kl_loss = -0.5 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
    
    return (reconst_loss + kl_loss)


# VAE model statement
vae = Model(input_img, pred)
vae.add_loss(vae_loss(input_img,pred))
optimizer = Adam(learning_rate=LEARNING_RATES[1])
vae.compile(optimizer=optimizer, loss=None)

vae.summary()


# fit and track the model 
fit_model_and_log_metrics("Model vae_5", "With maxpooling, convtranspose, dense, and flatten", vae, 50, 128)

Model: "model_23"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_20 (Conv2D)             (None, 16, 16, 32)   896         ['input_15[0][0]']               
                                                                                                  
 batch_normalization_27 (BatchN  (None, 16, 16, 32)  128         ['conv2d_20[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_21 (Conv2D)             (None, 8, 8, 16)     4624        ['batch_normalization_27[0

Epoch 1/50
391/391 [==============================] - 7s 16ms/step - loss: 11.9686 - val_loss: 16.9724 - _timestamp: 1648824486.0000 - _runtime: 14.0000
Epoch 2/50
391/391 [==============================] - 6s 14ms/step - loss: 5.8065 - val_loss: 6.4966 - _timestamp: 1648824492.0000 - _runtime: 20.0000
Epoch 3/50
391/391 [==============================] - 5s 14ms/step - loss: 4.9782 - val_loss: 4.9039 - _timestamp: 1648824497.0000 - _runtime: 25.0000
Epoch 4/50
391/391 [==============================] - 5s 13ms/step - loss: 4.5535 - val_loss: 4.7385 - _timestamp: 1648824503.0000 - _runtime: 31.0000
Epoch 5/50
391/391 [==============================] - 5s 13ms/step - loss: 4.2901 - val_loss: 4.2794 - _timestamp: 1648824508.0000 - _runtime: 36.0000
Epoch 6/50
391/391 [==============================] - 5s 14ms/step - loss: 4.1429 - val_loss: 4.6573 - _timestamp: 1648824513.0000 - _runtime: 41.0000
Epoch 7/50
391/391 [==============================] - 6s 14ms/step - loss: 4.0452 - val_loss

avg_ssim_score_x_test,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▂▂▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁
avg_ssim_score_x_test,0.91481
best_epoch,34
best_val_loss,3.45737
epoch,44
loss,3.41639
val_loss,3.56259


## Exp 5.1

In [ ]:
'''
Model vae_5.1
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

# x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
# x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
# print(shape_before_flattening)
x = Flatten()(x)

z_mu = Dense(latent_dim)(x)
z_log_sigma = Dense(latent_dim, kernel_initializer='zeros', bias_initializer='zeros')(x)


# sampling function
def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon

# sample vector from the latent distribution
z = Lambda(sampling)([z_mu, z_log_sigma])

encoder = Model(input_img,z)

# decoder takes the latent distribution sample as input
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(4096, activation='relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
x = Reshape((8,8,64))(x)

x = Conv2DTranspose(32, (3, 3),strides= 2, padding='same')(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(16, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2DTranspose(3, (3, 3),strides= 2, padding='same', activation='sigmoid')(x)


# decoder model statement
decoder = Model(decoder_input, x)

# apply the decoder to the sample from the latent distribution
pred = decoder(z)


def vae_loss(x, pred):
    x = K.flatten(x)
    pred = K.flatten(pred)
    # Reconstruction loss
    #reconst_loss = 100 * binary_crossentropy(x, pred)
    reconst_loss = 1000*K.mean(K.square(x - pred))
    
    # KL divergence
    kl_loss = -0.5 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
    
    return (reconst_loss + kl_loss)


# VAE model statement
vae = Model(input_img, pred)
vae.add_loss(vae_loss(input_img,pred))
optimizer = Adam(learning_rate=LEARNING_RATES[1])
vae.compile(optimizer=optimizer, loss=None)

vae.summary()


# fit and track the model 
fit_model_and_log_metrics("Model vae_5.1", "With maxpooling, convtranspose, dense, and flatten", vae, 50, 64)

Model: "model_26"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_17 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_22 (Conv2D)             (None, 16, 16, 32)   896         ['input_17[0][0]']               
                                                                                                  
 batch_normalization_30 (BatchN  (None, 16, 16, 32)  128         ['conv2d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_23 (Conv2D)             (None, 8, 8, 16)     4624        ['batch_normalization_30[0

Epoch 1/50
782/782 [==============================] - 10s 12ms/step - loss: 9.9584 - val_loss: 7.0512 - _timestamp: 1648824755.0000 - _runtime: 16.0000
Epoch 2/50
782/782 [==============================] - 8s 10ms/step - loss: 5.4466 - val_loss: 5.2877 - _timestamp: 1648824763.0000 - _runtime: 24.0000
Epoch 3/50
782/782 [==============================] - 8s 10ms/step - loss: 4.7412 - val_loss: 4.8593 - _timestamp: 1648824771.0000 - _runtime: 32.0000
Epoch 4/50
782/782 [==============================] - 8s 10ms/step - loss: 4.4083 - val_loss: 4.9576 - _timestamp: 1648824779.0000 - _runtime: 40.0000
Epoch 5/50
782/782 [==============================] - 8s 10ms/step - loss: 4.2472 - val_loss: 4.3547 - _timestamp: 1648824786.0000 - _runtime: 47.0000
Epoch 6/50
782/782 [==============================] - 8s 10ms/step - loss: 4.1321 - val_loss: 4.0545 - _timestamp: 1648824794.0000 - _runtime: 55.0000
Epoch 7/50
782/782 [==============================] - 8s 10ms/step - loss: 4.0754 - val_loss:

avg_ssim_score_x_test,▁
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▄▄▃▂▃▃▂▂▂▁▂▁▂▂▂▁▁▁▁▁▁▁▂▂▂▁▂▃
avg_ssim_score_x_test,0.91023
best_epoch,19
best_val_loss,3.57941
epoch,29
loss,3.54039
val_loss,4.33431


## Exp 6

In [ ]:
'''
Model vae_6
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='leaky_relu', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='leaky_relu', padding='same', strides=2)(x)
x = BatchNormalization()(x)

# x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
# x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
# print(shape_before_flattening)
x = Flatten()(x)

z_mu = Dense(latent_dim)(x)
z_log_sigma = Dense(latent_dim, kernel_initializer='zeros', bias_initializer='zeros')(x)


# sampling function
def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon

# sample vector from the latent distribution
z = Lambda(sampling)([z_mu, z_log_sigma])

encoder = Model(input_img,z)

# decoder takes the latent distribution sample as input
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(4096, activation='leaky_relu', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
x = Reshape((8,8,64))(x)

x = Conv2DTranspose(32, (3, 3),strides= 2, padding='same', activation="leaky_relu")(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(16, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2DTranspose(3, (3, 3),strides= 2, padding='same', activation='sigmoid')(x)


# decoder model statement
decoder = Model(decoder_input, x)

# apply the decoder to the sample from the latent distribution
pred = decoder(z)


def vae_loss(x, pred):
    x = K.flatten(x)
    pred = K.flatten(pred)
    # Reconstruction loss
    #reconst_loss = 100 * binary_crossentropy(x, pred)
    reconst_loss = 1000*K.mean(K.square(x - pred))
    
    # KL divergence
    kl_loss = -0.5 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
    
    return (reconst_loss + kl_loss)


# VAE model statement
vae = Model(input_img, pred)
vae.add_loss(vae_loss(input_img,pred))
optimizer = Adam(learning_rate=LEARNING_RATES[1])
vae.compile(optimizer=optimizer, loss=None)

vae.summary()


# fit and track the model 
fit_model_and_log_metrics("Model vae_6", "With maxpooling, convtranspose, dense, and flatten", vae, 50, 128)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 16, 16, 32)   896         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 16, 16, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 conv2d_1 (Conv2D)              (None, 8, 8, 16)     4624        ['batch_normalization[0][0]

wandb: Currently logged in as: csc2231-yash-siphelele (use `wandb login --relogin` to force relogin)


Epoch 1/50
391/391 [==============================] - 19s 19ms/step - loss: 11.9704 - val_loss: 14.7666 - _timestamp: 1648827686.0000 - _runtime: 29.0000
Epoch 2/50
391/391 [==============================] - 7s 17ms/step - loss: 5.7085 - val_loss: 5.1362 - _timestamp: 1648827693.0000 - _runtime: 36.0000
Epoch 3/50
391/391 [==============================] - 6s 16ms/step - loss: 4.6533 - val_loss: 4.7068 - _timestamp: 1648827699.0000 - _runtime: 42.0000
Epoch 4/50
391/391 [==============================] - 6s 16ms/step - loss: 4.2408 - val_loss: 4.1226 - _timestamp: 1648827706.0000 - _runtime: 49.0000
Epoch 5/50
391/391 [==============================] - 6s 16ms/step - loss: 4.0240 - val_loss: 4.2221 - _timestamp: 1648827712.0000 - _runtime: 55.0000
Epoch 6/50
391/391 [==============================] - 6s 16ms/step - loss: 3.8953 - val_loss: 3.8542 - _timestamp: 1648827718.0000 - _runtime: 61.0000
Epoch 7/50
391/391 [==============================] - 6s 15ms/step - loss: 3.8209 - val_los

avg_ssim_score_x_test,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_ssim_score_x_test,0.92001
best_epoch,43
best_val_loss,3.31661
epoch,49
loss,3.31158
val_loss,3.61067


## Exp 6.1

In [ ]:
'''
Model vae_6.1
'''

latent_dim = Z_DIM[2]  # Number of latent dimension parameters

input_img = Input(shape=(32, 32, 3))

x = Conv2D(32, (3, 3), activation='tanh', padding='same', strides=2)(input_img)
x = BatchNormalization()(x)

x = Conv2D(16, (3, 3), activation='tanh', padding='same', strides=2)(x)
x = BatchNormalization()(x)

# x = Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)(x)
# x = BatchNormalization()(x)

shape_before_flattening = K.int_shape(x)
# print(shape_before_flattening)
x = Flatten()(x)

z_mu = Dense(latent_dim)(x)
z_log_sigma = Dense(latent_dim, kernel_initializer='zeros', bias_initializer='zeros')(x)


# sampling function
def sampling(args):
    z_mu, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mu + K.exp(z_log_sigma) * epsilon

# sample vector from the latent distribution
z = Lambda(sampling)([z_mu, z_log_sigma])

encoder = Model(input_img,z)

# decoder takes the latent distribution sample as input
decoder_input = Input(K.int_shape(z)[1:])
x = Dense(4096, activation='tanh', name="intermediate_decoder", input_shape=(latent_dim,))(decoder_input)
x = Reshape((8,8,64))(x)

x = Conv2DTranspose(32, (3, 3),strides= 2, padding='same', activation="tanh")(x)
x = BatchNormalization()(x)

# x = Conv2DTranspose(16, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2DTranspose(8, (3, 3),strides= 2, padding='same')(x)
# x = BatchNormalization()(x)

x = Conv2DTranspose(3, (3, 3),strides= 2, padding='same', activation='sigmoid')(x)


# decoder model statement
decoder = Model(decoder_input, x)

# apply the decoder to the sample from the latent distribution
pred = decoder(z)


def vae_loss(x, pred):
    x = K.flatten(x)
    pred = K.flatten(pred)
    # Reconstruction loss
    #reconst_loss = 100 * binary_crossentropy(x, pred)
    reconst_loss = 1000*K.mean(K.square(x - pred))
    
    # KL divergence
    kl_loss = -0.5 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1)
    
    return (reconst_loss + kl_loss)


# VAE model statement
vae = Model(input_img, pred)
vae.add_loss(vae_loss(input_img,pred))
optimizer = Adam(learning_rate=LEARNING_RATES[1])
vae.compile(optimizer=optimizer, loss=None)

vae.summary()


# fit and track the model 
fit_model_and_log_metrics("Model vae_6.1", "With maxpooling, convtranspose, dense, and flatten", vae, 50, 128)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_2 (Conv2D)              (None, 16, 16, 32)   896         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 16, 16, 32)  128         ['conv2d_2[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 conv2d_3 (Conv2D)              (None, 8, 8, 16)     4624        ['batch_normalization_3[0][

Epoch 1/50
391/391 [==============================] - 9s 19ms/step - loss: 10.7063 - val_loss: 19.9280 - _timestamp: 1648828345.0000 - _runtime: 19.0000
Epoch 2/50
391/391 [==============================] - 7s 18ms/step - loss: 5.3318 - val_loss: 7.2817 - _timestamp: 1648828352.0000 - _runtime: 26.0000
Epoch 3/50
391/391 [==============================] - 6s 16ms/step - loss: 4.5892 - val_loss: 6.2973 - _timestamp: 1648828358.0000 - _runtime: 32.0000
Epoch 4/50
391/391 [==============================] - 6s 16ms/step - loss: 4.3596 - val_loss: 5.2993 - _timestamp: 1648828364.0000 - _runtime: 38.0000
Epoch 5/50
391/391 [==============================] - 6s 16ms/step - loss: 4.2394 - val_loss: 4.2469 - _timestamp: 1648828371.0000 - _runtime: 45.0000
Epoch 6/50
391/391 [==============================] - 6s 16ms/step - loss: 4.1532 - val_loss: 5.2460 - _timestamp: 1648828377.0000 - _runtime: 51.0000
Epoch 7/50
391/391 [==============================] - 6s 15ms/step - loss: 4.0943 - val_loss

avg_ssim_score_x_test,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
loss,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_ssim_score_x_test,0.91276
best_epoch,30
best_val_loss,3.65163
epoch,40
loss,3.58094
val_loss,3.81913
